<a href="https://colab.research.google.com/github/VitalyGladyshev/med_work/blob/main/HD_HW_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Прогнозирование и диагностика в медицине с помощью машинного обучения

## ДЗ 2 Гладышев В.В.

## Модели классификации

### Импорты

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score

from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
RANDOM_STATE = 42

### Загрузка и подготовка данных

In [4]:
!unzip ct_slices.zip

Archive:  ct_slices.zip
replace slice_localization_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [4]:
df = pd.read_csv("/content/slice_localization_data.csv")

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53500 entries, 0 to 53499
Columns: 386 entries, patientId to reference
dtypes: float64(385), int64(1)
memory usage: 157.6 MB


In [6]:
df.head()

,patientId,value0,value1,value2,value3,value4,value5,value6,value7,value8,...,value375,value376,value377,value378,value379,value380,value381,value382,value383,reference
0,0,0.0,0.0,0.0,0.0,0.0,0.0,-0.25,-0.25,-0.25,...,-0.25,0.980381,0.0,0.0,0.0,0.0,0.0,-0.25,-0.25,21.803851
1,0,0.0,0.0,0.0,0.0,0.0,0.0,-0.25,-0.25,-0.25,...,-0.25,0.977008,0.0,0.0,0.0,0.0,0.0,-0.25,-0.25,21.745726
2,0,0.0,0.0,0.0,0.0,0.0,0.0,-0.25,-0.25,-0.25,...,-0.25,0.977008,0.0,0.0,0.0,0.0,0.0,-0.25,-0.25,21.687600
3,0,0.0,0.0,0.0,0.0,0.0,0.0,-0.25,-0.25,-0.25,...,-0.25,0.977008,0.0,0.0,0.0,0.0,0.0,-0.25,-0.25,21.629474
4,0,0.0,0.0,0.0,0.0,0.0,0.0,-0.25,-0.25,-0.25,...,-0.25,0.976833,0.0,0.0,0.0,0.0,0.0,-0.25,-0.25,21.571348


In [7]:
df.describe()

,patientId,value0,value1,value2,value3,value4,value5,value6,value7,value8,...,value375,value376,value377,value378,value379,value380,value381,value382,value383,reference
count,53500.000000,53500.000000,53500.000000,53500.000000,53500.000000,53500.000000,53500.000000,53500.000000,53500.000000,53500.000000,...,53500.000000,53500.000000,53500.000000,53500.000000,53500.000000,53500.000000,53500.000000,53500.000000,53500.000000,53500.000000
mean,47.075701,0.059627,0.071558,0.145819,0.218728,0.274762,0.276189,0.204531,0.062281,-0.042025,...,-0.029404,0.182913,0.320112,0.359373,0.342889,0.266091,0.083049,-0.031146,-0.154524,47.028039
std,27.414240,0.174243,0.196921,0.300270,0.359163,0.378862,0.369605,0.351294,0.292232,0.268391,...,0.085817,0.383333,0.463517,0.478188,0.471811,0.437633,0.279734,0.098738,0.122491,22.347042
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.250000,-0.250000,-0.250000,-0.250000,...,-0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.250000,-0.250000,-0.250000,1.738733
25%,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.250000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.250000,29.891607
50%,46.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.250000,43.987893
75%,70.000000,0.000000,0.000000,0.000000,0.446429,0.684477,0.662382,0.441412,0.000000,0.000000,...,0.000000,0.000000,0.996286,0.999677,0.999560,0.949478,0.000000,0.000000,0.000000,63.735059
max,96.000000,1.000000,1.000000,1.000000,1.000000,0.998790,0.996468,0.999334,1.000000,1.000000,...,0.961279,1.000000,1.000000,1.000000,1.000000,1.000000,0.999857,0.996839,0.942851,97.489115


Проверка на пропуски

In [8]:
df.isna().sum()

,0
patientId,0
value0,0
value1,0
value2,0
value3,0
...,...
value380,0
value381,0
value382,0
value383,0


In [9]:
def data_filtering(data):
    colums_for_filtering = []

    for i in data.columns:
        amount_025 = list(data[i].values).count(-0.25) / len(data[i])
        if amount_025 > 0.75:
            colums_for_filtering.append(i)

    print(len(colums_for_filtering))

    data_filtered = data.drop(colums_for_filtering, axis = 1)

    return data_filtered

In [10]:
df_shuffle = df.sample(frac=1)

In [11]:
data_filtered = data_filtering(df_shuffle)
data_filtered.info()

44
<class 'pandas.core.frame.DataFrame'>
Index: 53500 entries, 1715 to 45051
Columns: 342 entries, patientId to reference
dtypes: float64(341), int64(1)
memory usage: 140.0 MB


In [12]:
df_shuffle.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53500 entries, 1715 to 45051
Columns: 386 entries, patientId to reference
dtypes: float64(385), int64(1)
memory usage: 158.0 MB


Разделяем данные для обучения

In [13]:
X = df_shuffle.drop('reference', axis=1)
y = df_shuffle.reference.round()

X_f = data_filtered.drop('reference', axis=1)
y_f = data_filtered.reference.round()

In [14]:
y.unique().shape

(96,)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_STATE)
X_f_train, X_f_test, y_f_train, y_f_test = train_test_split(X_f, y_f, test_size=0.3, random_state=RANDOM_STATE)

In [16]:
X.head()

,patientId,value0,value1,value2,value3,value4,value5,value6,value7,value8,...,value374,value375,value376,value377,value378,value379,value380,value381,value382,value383
1715,3,0.0,0.000000,0.000000,0.000000,0.805455,0.949894,0.000000,0.880435,-0.25,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,-0.25
45436,82,0.0,0.000000,0.980133,0.984241,0.902298,0.000000,0.000000,0.000000,0.00,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00
49983,89,0.0,0.000000,0.000000,0.000000,0.113565,0.910934,0.402516,0.000000,0.00,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00
47094,85,0.0,0.000000,0.000000,0.000000,0.000000,0.562805,0.000000,0.000000,0.00,...,0.0,0.0,0.0,0.999537,0.999991,0.999996,0.998724,0.0,0.0,-0.25
18360,33,0.0,0.754557,0.041026,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00


### Сравнение алгоритмов классификации

In [17]:
def classification_metrics(y, pred):
    pre = accuracy_score(y, pred)
    print(f"accuracy score:\t\t{pre:.5f}\n")

    pre = f1_score(y, pred, average="macro")
    print(f"f1 score:\t\t{pre:.5f}")

    pre = precision_score(y, pred, average="macro")
    print(f"precision score:\t{pre:.5f}")

    pre = recall_score(y, pred, average="macro", zero_division=0.0)
    print(f"recall score:\t\t{pre:.5f}")

### DecisionTreeClassifier

#### Без фильтрации

In [19]:
dt_model = DecisionTreeClassifier(max_depth=7, random_state=RANDOM_STATE)
dt_model.fit(X_train, y_train)
preds = dt_model.predict(X_test)
classification_metrics(y_test, preds)

accuracy score:		0.25059

f1 score:		0.18678
precision score:	0.20224
recall score:		0.22180


In [20]:
decision_tree_classifier = make_pipeline(DecisionTreeClassifier(max_depth=7, random_state=RANDOM_STATE))
dtc_rmses = cross_val_score(decision_tree_classifier, X, y, cv=5)
print(f"{pd.Series(dtc_rmses).describe()['mean']:.4f} {dtc_rmses}")

0.2592 [0.25925234 0.26       0.26542056 0.25476636 0.25654206]


#### С фильтрацией

In [21]:
dt_f_model = DecisionTreeClassifier(max_depth=7, random_state=RANDOM_STATE)
dt_f_model.fit(X_f_train, y_f_train)
preds = dt_f_model.predict(X_f_test)
classification_metrics(y_f_test, preds)

accuracy score:		0.25065

f1 score:		0.18683
precision score:	0.20233
recall score:		0.22180


In [22]:
decision_tree_classifier_f = make_pipeline(DecisionTreeClassifier(max_depth=7, random_state=RANDOM_STATE))
dtc_rmses_f = cross_val_score(decision_tree_classifier_f, X_f, y_f, cv=5)
print(f"{pd.Series(dtc_rmses_f).describe()['mean']:.4f} {dtc_rmses_f}")

0.2593 [0.25962617 0.26       0.26542056 0.25476636 0.25654206]


### RandomForestClassifier

#### Без фильтрации

In [23]:
rf_model = RandomForestClassifier(random_state=RANDOM_STATE)
rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)
classification_metrics(y_test, preds)

accuracy score:		0.90199

f1 score:		0.90066
precision score:	0.90311
recall score:		0.90049


In [21]:
forest_classifier = make_pipeline(RandomForestClassifier(random_state=RANDOM_STATE))
forest_rmses = cross_val_score(forest_classifier, X, y, cv=5)
print(f"{pd.Series(forest_rmses).describe()['mean']:.4f} {forest_rmses}")

0.9076 [0.90757009 0.90654206 0.90990654 0.90747664 0.90635514]


#### С фильтрацией

In [25]:
rf_model_f = RandomForestClassifier(random_state=RANDOM_STATE)
rf_model_f.fit(X_f_train, y_f_train)
preds = rf_model_f.predict(X_f_test)
classification_metrics(y_f_test, preds)

accuracy score:		0.90056

f1 score:		0.89898
precision score:	0.90116
recall score:		0.89888


In [22]:
forest_classifier_f = make_pipeline(RandomForestClassifier(random_state=RANDOM_STATE))
forest_rmses_f = cross_val_score(forest_classifier_f, X_f, y_f, cv=5)
print(f"{pd.Series(forest_rmses_f).describe()['mean']:.4f} {forest_rmses_f}")

0.9084 [0.90971963 0.90728972 0.9088785  0.90990654 0.9064486 ]


### Support Vector Machine

#### Без фильтрации

In [27]:
svm_model = SVC(gamma="auto", random_state=RANDOM_STATE)
svm_model.fit(X_train, y_train)
preds = svm_model.predict(X_test)
classification_metrics(y_test, preds)

accuracy score:		0.41533

f1 score:		0.35461
precision score:	0.37474
recall score:		0.36403


#### С фильтрацией

In [19]:
svm_model_f = SVC(kernel='linear', random_state=RANDOM_STATE)
svm_model_f.fit(X_f_train, y_f_train)
preds = svm_model_f.predict(X_f_test)
classification_metrics(y_f_test, preds)

accuracy score:		0.81109

f1 score:		0.79485
precision score:	0.79905
recall score:		0.80081


### Получение прогноза

In [23]:
pred = cross_val_predict(forest_classifier, X, y, cv=5)

In [24]:
classification_metrics(y, pred)

accuracy score:		0.90757

f1 score:		0.90433
precision score:	0.90517
recall score:		0.90429


In [25]:
pred_f = cross_val_predict(forest_classifier_f, X_f, y_f, cv=5)

In [27]:
classification_metrics(y_f, pred_f)

accuracy score:		0.90845

f1 score:		0.90480
precision score:	0.90567
recall score:		0.90470


### Заключение

Лучшее значение метрики показал алгоритм RandomForestClassifier на наборе данных с фильтрацией при обучении методом кроссвалидации